In [2]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.lines as mlines

# === 样式设置 ===
plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['axes.titlesize'] = 52
plt.rcParams['axes.labelsize'] = 52
plt.rcParams['lines.linewidth'] = 8
plt.rcParams['xtick.labelsize'] = 52
plt.rcParams['ytick.labelsize'] = 52
plt.rcParams['font.family'] = 'serif'
plt.rcParams['legend.fontsize'] = 42
plt.rcParams['figure.dpi'] = 600

# === 策略缩写映射 ===
policy_display_map = {
    "fcfs": "vLLM",
    "fcfs_mc": "Mooncake",
    "ldf_rej_gate1.3_credit": "Scorpio(Ours)",
    "ssjf_pred": "S3"
}

# === 样式映射 ===
policy_style_map = {
    "Scorpio(Ours)": {'color': '#ec6446', 'marker': 's', 'linestyle': '-', 'zorder': 10},
    "vLLM": {'color': '#f29d46', 'marker': '^', 'linestyle': '--', 'zorder': 8},
    "Mooncake": {'color': '#3f906e', 'marker': 'D', 'linestyle': '--', 'zorder': 6},
    "S3": {'color': '#836ed5', 'marker': 'o', 'linestyle': '-', 'zorder': 4},
}

# === 时间 bucket 间隔 ===
bucket_interval = '10s'

# === 主图绘制 ===
plt.figure()

for filename in sorted(os.listdir(".")):
    if not filename.endswith(".json"):
        continue

    base = filename.split("_qps")[0].replace("r1_", "")
    policy_name = policy_display_map.get(base)
    if policy_name is None:
        continue

    with open(filename, "r") as f:
        data = json.load(f)

    records = data["trace_slo_adherence"]
    df = pd.DataFrame(records, columns=["timestamp", "slo_met"])
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df.set_index("timestamp", inplace=True)

    bucketed = df.resample(bucket_interval).agg(slo_met_count=("slo_met", "sum")).fillna(0)
    bucketed["cumulative_slo_met"] = bucketed["slo_met_count"].cumsum()
    relative_minutes = (bucketed.index - bucketed.index.min()).total_seconds() / 60

    style = policy_style_map[policy_name]
    plt.plot(
        relative_minutes,
        bucketed["cumulative_slo_met"],
        label=policy_name,
        color=style['color'],
        marker=style['marker'],
        linestyle=style['linestyle'],
        linewidth=8,
        markersize=32,
        markevery=10,  # 每隔10个点画一个marker，可根据曲线长度调整
        zorder=style['zorder']
    )
    from matplotlib.ticker import FuncFormatter
    def thousands_formatter(x, pos):
        return f'{int(x/1000)}k' if x >= 1000 else str(int(x))

    plt.gca().yaxis.set_major_formatter(FuncFormatter(thousands_formatter))


# === 主图保存 ===
plt.xlabel("Timeline (min)")
plt.ylabel("Cum. # SLO Met")
plt.grid(True)
plt.tight_layout()
plt.savefig("slo_met_over_time_cdf.pdf", dpi=600, bbox_inches='tight', format='pdf')
plt.close()

# === 单独绘制 legend 图 ===
labels = ['Scorpio(Ours)', 'vLLM', 'Mooncake', 'S3']
colors = ['#ec6446', '#f29d46', '#3f906e', '#836ed5']
markers = ['s', '^', 'D', 'o']
linestyles = ['-', '--', '--', '-']
linewidth = 6
legend_fontsize = 52

fig, ax = plt.subplots(figsize=(25, 1))
handles = [
    mlines.Line2D(
        [], [], color=colors[i], marker=markers[i], linestyle=linestyles[i],
        linewidth=linewidth, label=label, markersize=42
    )
    for i, label in enumerate(labels)
]

ax.legend(handles=handles, loc='center', fontsize=legend_fontsize, ncol=len(labels), frameon=False)
ax.axis('off')
plt.savefig("slo_goodput_legend.pdf", dpi=600, format="pdf", bbox_inches="tight")
plt.close()
